# 🤖 Open Source Contribution Agent - Full Pipeline Demo

This notebook demonstrates the complete multi-agent pipeline for autonomous open source contributions with **real GitHub integration**.

## Agent Pipeline Overview

1. **🔍 Scout Agent** - Discovers suitable repositories
2. **📊 Analyzer Agent** - Analyzes repositories for contribution opportunities
3. **🎯 Strategist Agent** - Plans optimal contribution strategy
4. **💻 Coder Agent** - Generates code, documentation, and PR drafts
5. **🚀 Real Implementation** - Actually creates forks, branches, and Pull Requests on GitHub

Let's see the agents in action!

## Setup and Configuration

First, let's import our agents and set up the environment.

In [ ]:
import os
import sys
sys.path.append('..')

# Import our agents
from agents.scout import ScoutAgent
from agents.analyzer import AnalyzerAgent
from agents.strategist import StrategistAgent
from agents.coder import CoderAgent
from core.utils import load_config

from rich.console import Console
from rich.panel import Panel
import json

# Initialize console for pretty output
console = Console()

# Load configuration (GitHub token, Anthropic key)
config = load_config()

console.print(Panel(
    "[bold green]🚀 Open Source Contribution Agent initialized![/bold green]\n\n"
    "This demo shows our multi-agent system that discovers repositories,\n"
    "analyzes contribution opportunities, and creates real GitHub PRs.",
    title="Welcome to the AI Agent Demo"
))

## Step 1: 🔍 Repository Discovery (Scout Agent)

The Scout Agent searches GitHub for repositories that match our contribution criteria.

In [ ]:
# Initialize Scout Agent
scout = ScoutAgent(github_token=config.get('GITHUB_TOKEN'))

console.print(Panel(
    "[bold green]🔍 Discovering Repositories Using Your Preferences[/bold green]\n\n"
    "The Scout Agent will automatically use your saved preferences:\n"
    f"• Your programming languages\n"
    f"• Your star range preferences\n"
    f"• Your project size preferences\n\n"
    f"[dim]Loading from data/user_preferences.json[/dim]",
    title="🤖 Automated Repository Discovery"
))

# Discover repositories - agent will auto-load YOUR preferences
discovered_repos = scout.discover_repositories(
    max_results=10  # Only specify what's not in preferences
)

# Display results
scout.display_repositories(discovered_repos)

# Save discovered repositories for other agents
scout.save_discovered_repos(discovered_repos)

print(f"\n✅ Found {len(discovered_repos)} suitable repositories for contribution!")

## Step 2: 📊 Repository Analysis (Analyzer Agent)

The Analyzer Agent performs deep analysis of the most promising repository to identify specific contribution opportunities.

In [ ]:
# Initialize Analyzer Agent
analyzer = AnalyzerAgent(
    github_token=config.get('GITHUB_TOKEN'),
    anthropic_api_key=config.get('ANTHROPIC_API_KEY')
)

# Select the most promising repository (highest scoring one)
if discovered_repos:
    target_repo = discovered_repos[0]  # Top-scored repository
    repo_quick_info = scout.get_repository_quick_info(target_repo)
    
    console.print(Panel(
        f"[bold cyan]{repo_quick_info['full_name']}[/bold cyan]\n"
        f"⭐ Stars: {repo_quick_info['stars']}\n"
        f"🔧 Language: {repo_quick_info['language']}\n"
        f"📊 Contribution Score: {repo_quick_info['contribution_score']}\n\n"
        f"Description: {repo_quick_info['description'] or 'No description'}",
        title="📊 Selected Repository for Analysis"
    ))
    
    # Perform comprehensive analysis
    analysis_result = analyzer.analyze_repository(repo_quick_info)
    
    # Display analysis results
    analyzer.display_analysis(analysis_result)
    
    print(f"\n✅ Analysis complete! Found {len(analysis_result['contribution_opportunities'])} opportunities.")
else:
    print("❌ No repositories found for analysis. Please check your search criteria.")

## Step 3: 🎯 Contribution Strategy (Strategist Agent)

The Strategist Agent creates a detailed plan for contributing to the selected repository.

In [ ]:
# Initialize Strategist Agent
strategist = StrategistAgent(anthropic_api_key=config.get('ANTHROPIC_API_KEY'))

console.print(Panel(
    "[bold green]🎯 Creating Strategy Using Your Preferences[/bold green]\n\n"
    "The Strategist Agent will automatically load your saved preferences from:\n"
    f"• data/user_preferences.json\n"
    f"• Your contribution type ratings (1-10 scale)\n"
    f"• Your available time and skill level\n"
    f"• Your programming language preferences\n\n"
    f"[dim]No manual preference definition needed![/dim]",
    title="🤖 Automated Strategy Planning"
))

# Create contribution strategy - agent will auto-load YOUR preferences
if 'analysis_result' in locals():
    strategy = strategist.create_contribution_strategy(analysis_result)

    # Display strategy
    strategist.display_strategy(strategy)

    # Save strategy for the Coder Agent
    strategist.save_strategy(strategy)

    print(f"\n✅ Strategic plan created! Success probability: {strategy['success_probability']:.1%}")
else:
    print("❌ No analysis results available for strategy creation.")

## Step 4: 💻 Code Generation (Coder Agent)

The Coder Agent generates actual code, documentation, tests, and PR templates for the highest-priority contribution opportunity.

In [ ]:
# Initialize Coder Agent
coder = CoderAgent(anthropic_api_key=config.get('ANTHROPIC_API_KEY'))

# Select the top opportunity from our strategy
if 'strategy' in locals() and strategy['contribution_plan']:
    top_opportunity = strategy['contribution_plan'][0]  # Highest priority item
    
    console.print(Panel(
        f"[bold green]{top_opportunity['opportunity']['title']}[/bold green]\n\n"
        f"Type: {top_opportunity['opportunity']['type'].title()}\n"
        f"Priority: {top_opportunity['opportunity']['priority'].title()}\n"
        f"Estimated Timeline: {top_opportunity['estimated_timeline']}\n"
        f"Effort: {top_opportunity['opportunity']['effort'].title()}\n"
        f"Impact: {top_opportunity['opportunity']['impact'].title()}",
        title="💻 Generating Code for Top Opportunity"
    ))
    
    # Generate complete contribution
    contribution = coder.generate_contribution(
        top_opportunity, 
        repo_quick_info
    )
    
    # Display generated contribution
    coder.display_contribution(contribution)
    
    # Save the generated contribution
    coder.save_contribution(contribution)
    
    print(f"\n✅ Complete contribution generated with PR template and implementation!")
else:
    print("❌ No strategic plan available for code generation.")

## Step 5: 🚀 Real Implementation

The Real Implementation step takes the output from Step 4 and actually implements it by making real GitHub changes with Docker-based testing.

In [ ]:
# Step 5: Real Implementation - Actually make the GitHub changes!
if 'contribution' in locals() and 'repo_quick_info' in locals():
    console.print(Panel(
        "[bold green]🚀 Ready to Make Real Changes![/bold green]\n\n"
        "This enhanced implementation will:\n"
        "• Clone the repository safely\n"
        "• Create a feature branch\n"
        "• Apply the changes from Step 4\n"
        "• Generate unit tests (for code changes)\n"
        "• Test changes in Docker container\n"
        "• Fork the repository\n"
        "• Push changes and create a Pull Request\n\n"
        "[yellow]⚠️  SIMULATION MODE: Creates test PR on your own repo!\n"
        "Safe for testing - won't affect the original repository.[/yellow]",
        title="🧪 Simulation Mode - Safe Testing"
    ))

    # Actually implement the changes using the enhanced Coder Agent
    result = coder.implement_contribution_real(
        contribution, 
        repo_quick_info, 
        config.get('GITHUB_TOKEN'),
        simulate=True
    )

    # Display results - handle both simulation and real modes
    if result['status'] in ['success', 'simulation_success']:
        if result['status'] == 'simulation_success':
            console.print(Panel(
                f"[bold green]🧪 SIMULATION SUCCESS![/bold green]\n\n"
                f"✅ Test PR created: [link]{result['pr_url']}[/link]\n"
                f"🎯 Original repo: {result['original_repo']}\n"
                f"🧪 Test repo: {result['test_repo']}\n"
                f"🌿 Branch: {result['branch_name']}\n"
                f"📝 Commit: {result['commit_hash'][:8]}\n\n"
                f"[cyan]{result['note']}[/cyan]",
                title="🏆 Simulation Complete"
            ))
        else:
            console.print(Panel(
                f"[bold green]🎉 SUCCESS![/bold green]\n\n"
                f"✅ Pull Request created: [link]{result['pr_url']}[/link]\n"
                f"🌿 Branch: {result['branch_name']}\n"
                f"📝 Commit: {result['commit_hash'][:8]}\n"
                f"🍴 Fork: [link]{result['fork_url']}[/link]\n\n"
                f"[cyan]Your AI agent created a real contribution with tests! 🚀[/cyan]",
                title="🏆 Implementation Complete"
            ))

        # Show what was changed
        console.print("\n[bold]📁 Files modified/created:[/bold]")
        for file_path in contribution.get('files_to_modify', []):
            console.print(f"  📝 Modified: {file_path}")
        for file_path in contribution.get('files_to_create', []):
            console.print(f"  📄 Created: {file_path}")

        print(f"\n🎯 Review the PR: {result['pr_url']}")

    elif result['status'] == 'simulation_error':
        console.print(Panel(
            f"[bold red]❌ Simulation Failed[/bold red]\n\n"
            f"Error: {result['error']}\n\n"
            f"Workspace: {result.get('workspace', 'N/A')}",
            title="💥 Simulation Error"
        ))
    else:
        console.print(Panel(
            f"[bold red]❌ Implementation Failed[/bold red]\n\n"
            f"Error: {result['error']}\n"
            f"Test Output: {result.get('test_output', 'N/A')}\n\n"
            f"Workspace: {result.get('workspace', 'N/A')}",
            title="💥 Error"
        ))

else:
    print("❌ No contribution from Step 4 available. Run the previous steps first.")

In [ ]:
# Create comprehensive pipeline summary
if all(var in locals() for var in ['discovered_repos', 'analysis_result', 'strategy', 'contribution']):
    # Check if we have implementation results from Step 5
    implementation_results = {}
    if 'result' in locals():
        implementation_results = {
            'status': result['status'],
            'pr_url': result.get('pr_url', 'N/A'),
            'branch_name': result.get('branch_name', 'N/A'),
            'fork_url': result.get('fork_url', 'N/A'),
            'real_pr_created': result['status'] == 'success'
        }
    else:
        implementation_results = {
            'status': 'not_run',
            'pr_url': 'N/A',
            'branch_name': 'N/A',
            'fork_url': 'N/A',
            'real_pr_created': False
        }
    
    pipeline_summary = {
        'scout_results': {
            'repositories_found': len(discovered_repos),
            'selected_repository': repo_quick_info['full_name'],
            'selection_score': repo_quick_info['contribution_score']
        },
        'analysis_results': {
            'health_score': analysis_result['health_score'],
            'opportunities_found': len(analysis_result['contribution_opportunities']),
            'readme_quality': analysis_result['readme_analysis']['quality_score'],
            'good_first_issues': analysis_result['issues_analysis']['good_first_issues']
        },
        'strategy_results': {
            'success_probability': strategy['success_probability'],
            'planned_contributions': len(strategy['contribution_plan']),
            'risk_level': strategy['risk_assessment']['risk_level'],
            'recommendations_count': len(strategy['recommendations'])
        },
        'coder_results': {
            'contribution_type': contribution['type'],
            'files_to_create': len(contribution.get('files_to_create', [])),
            'files_to_modify': len(contribution.get('files_to_modify', [])),
            'pr_ready': bool(contribution['pr_template'])
        },
        'implementation_results': implementation_results
    }
    
    # Display pipeline summary
    step5_status = "✅ Completed" if implementation_results['real_pr_created'] else ("❌ Failed" if implementation_results['status'] != 'not_run' else "⏭️ Not Run")
    
    console.print(Panel(
        f"[bold green]🎉 MULTI-AGENT PIPELINE RESULTS! 🎉[/bold green]\n\n"
        f"**🔍 Scout Agent:**\n"
        f"• Found {pipeline_summary['scout_results']['repositories_found']} repositories\n"
        f"• Selected: {pipeline_summary['scout_results']['selected_repository']}\n\n"
        f"**📊 Analyzer Agent:**\n"
        f"• Repository health: {pipeline_summary['analysis_results']['health_score']}/100\n"
        f"• Found {pipeline_summary['analysis_results']['opportunities_found']} opportunities\n\n"
        f"**🎯 Strategist Agent:**\n"
        f"• Success probability: {pipeline_summary['strategy_results']['success_probability']:.1%}\n"
        f"• Risk level: {pipeline_summary['strategy_results']['risk_level']}\n\n"
        f"**💻 Coder Agent:**\n"
        f"• Generated {pipeline_summary['coder_results']['contribution_type']} contribution\n"
        f"• Created {pipeline_summary['coder_results']['files_to_create']} new files\n"
        f"• Modified {pipeline_summary['coder_results']['files_to_modify']} files\n\n"
        f"**🚀 Real Implementation:**\n"
        f"• Status: {step5_status}\n"
        f"• Branch: {pipeline_summary['implementation_results']['branch_name']}\n"
        f"• PR URL: {pipeline_summary['implementation_results']['pr_url']}",
        title="🚀 Multi-Agent Pipeline with Real GitHub Integration"
    ))
    
    # Save complete pipeline results
    with open('../data/pipeline_results.json', 'w') as f:
        json.dump(pipeline_summary, f, indent=2, default=str)
    
    console.print("\n💾 **Complete results saved to `data/pipeline_results.json`**")
    
    # Show success message if Step 5 worked
    if implementation_results['real_pr_created']:
        console.print(Panel(
            f"[bold green]🎯 SUCCESS! Real PR Created:[/bold green]\n\n"
            f"🔗 **PR URL:** {implementation_results['pr_url']}\n"
            f"🍴 **Your Fork:** {implementation_results['fork_url']}\n"
            f"🌿 **Branch:** {implementation_results['branch_name']}\n\n"
            f"[cyan]Your AI agent made a real open source contribution![/cyan]\n"
            f"The repository owner can now review and merge your PR.",
            title="🏆 Real Implementation Success"
        ))
    elif implementation_results['status'] == 'not_run':
        console.print(Panel(
            f"[bold yellow]ℹ️  Step 5 Not Run Yet[/bold yellow]\n\n"
            f"Re-run Step 5 after restarting your kernel to see the real\n"
            f"GitHub integration create actual Pull Requests!",
            title="📝 Next Steps"
        ))
    
else:
    missing_components = []
    if 'discovered_repos' not in locals(): missing_components.append('Scout results')
    if 'analysis_result' not in locals(): missing_components.append('Analysis results')
    if 'strategy' not in locals(): missing_components.append('Strategy results') 
    if 'contribution' not in locals(): missing_components.append('Code generation results')
    
    console.print(Panel(
        f"[bold red]⚠️ Pipeline Incomplete[/bold red]\n\n"
        f"Missing components: {', '.join(missing_components)}\n\n"
        f"Please run all previous cells to see the complete pipeline in action.",
        title="❌ Incomplete Pipeline"
    ))

## 🎯 Key Features & Capabilities

This multi-agent system now provides **real GitHub integration** with advanced safety features:

### 🔥 **Enhanced Step 5 - Real Implementation:**
1. **🐳 Docker-based Testing** - All code runs in isolated containers for safety
2. **🧪 Automatic Test Generation** - Creates unit tests for bug fixes and new features  
3. **🔍 Framework Detection** - Automatically detects pytest, jest, cargo, go test, etc.
4. **🍴 Smart Fork Management** - Creates forks since we can't push to others' repositories
5. **🚀 Real PR Creation** - Actually creates Pull Requests on GitHub

### 💼 **Production-Ready Features:**
- **Multi-language support**: Python, JavaScript, TypeScript, Rust, Go
- **Intelligent analysis**: Repository health scoring, opportunity detection
- **User preferences**: Customizable contribution type preferences
- **Safety first**: Docker isolation prevents untrusted code execution
- **Test automation**: Generates and runs tests before creating PRs

### 🎯 **Perfect for AI Startups:**
This demonstrates advanced **multi-agent coordination**, **real-world integration**, and **production safety** - exactly what AI companies are looking for!

---

**Ready to create real open source contributions with AI agents? Run the cells above! 🚀**